# SWEs in 1D with Non-Linear Problem
## Problem Formulation Link:
https://github.com/partha-sakha-paul/FEniCS_Landlab/blob/main/weak%20form%20SWEs.pdf

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx import mesh, fem
from dolfinx.fem import Function, Constant, dirichletbc, locate_dofs_geometrical
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.fem.petsc import NonlinearProblem
from ufl import TrialFunction, TestFunction, dx, grad

# Define parameters
Lx = 7000.0           # Domain length
nx = 25            # Number of elements
dt = 0.1           # Time step
T = 9000.0
num_steps = int(T / dt)     # Number of time steps
u_const = 0.635       # Constant wave speed
n_value = 0.005            # Manning's coefficient

# Create 1D mesh and function space
domain = mesh.create_interval(MPI.COMM_WORLD, nx, [0, Lx])
V = fem.functionspace(domain, ("Lagrange", 1))

# Define functions
h = fem.Function(V)       # Current water depth
h_prev = fem.Function(V)  # Previous water depth

# Initialize h with zero (dry bed)
h.interpolate(lambda x: np.zeros_like(x[0]))

# Define test and trial functions
h_trial = TrialFunction(V)
v = TestFunction(V)

# Constants
u = Constant(domain, u_const)  # Velocity
n = Constant(domain, n_value)  # Manning's coefficient

# Weak form of continuity equation
# Integral Form:
# int (h_(n+1) - h_n) / dt v dx + int u delh/delx v dx
F1 = (h - h_prev) / dt * v * dx + u * h.dx(0) * v * dx

# Weak form of momentum equation
# Integral Form:
# int delh/delx v dx - n^2 u^2 / h^(4/3) dx
# F2 = h.dx(0) * v * dx - (n**2 * u**2)/h**(4/3)  * v * dx
F2 = h.dx(0) * v * dx - (n**2 * u**2) / h**(2) * v * dx

# Combine weak forms
F = F1 + F2

# Boundary conditions
def upstream_bc(t):
    return (7 / 3 * (n_value**2 * u_const**3 * t))**(3 / 7)

# Define boundary conditions
h_bc_up = fem.Constant(domain, PETSc.ScalarType(0.0))  # Upstream boundary value
# print(h_bc_up) # const func: c_something
h_bc_down = fem.Constant(domain, PETSc.ScalarType(0.0))  # Downstream boundary value
# print(h_bc_up)  # const func: c_something


# Locate degrees of freedom for the boundaries
dofs_upstream = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], 0.0))
print(h.x.array.shape)  # (26,)  total no of nodes
print(dofs_upstream)  # [0]  index of first element
dofs_downstream = locate_dofs_geometrical(V, lambda x: np.isclose(x[0], Lx))
print(dofs_downstream)  # [25]  index of last element


# Define boundary conditions
bc_upstream = fem.dirichletbc(h_bc_up, dofs_upstream, V)
bc_downstream = fem.dirichletbc(h_bc_down, dofs_downstream, V)

# Combine boundary conditions
bcs = [bc_upstream, bc_downstream]
bcs_up = [bc_upstream]


# Nonlinear problem setup
# problem = NonlinearProblem(F, h, bcs)  # taking both side bcs
problem = NonlinearProblem(F, h, bcs_up) # taking only upstream bc
solver = NewtonSolver(MPI.COMM_WORLD, problem)
# solver = NewtonSolver(domain.comm, problem)
solver.tolerance = 1e-6
solver.max_iterations = 250000

# Time-stepping loop
plot_steps = [0, num_steps // 4, num_steps // 2, 3 * num_steps // 4, num_steps - 1]  # Steps to plot
solutions = []

# Update boundary condition inside the loop
for step in range(num_steps):
    t = step * dt

    # Update upstream boundary condition
    h_bc_up.value = PETSc.ScalarType(upstream_bc(t))
    
    # Solve nonlinear problem
    solver.solve(h)

    # Save solution at specific steps for plotting
    if step in plot_steps:
        solutions.append(h.x.array.copy())

    # Update previous solution
    h_prev.x.array[:] = h.x.array[:]

# Plot results
x = domain.geometry.x[:, 0]
plt.figure(figsize=(10, 6))
for i, step in enumerate(plot_steps):
    plt.plot(x, solutions[i], label=f"Time step {step * dt:.2f} s")
plt.xlabel("x (m)")
plt.ylabel("h (m)")
plt.title("Water depth at different time steps")
plt.legend()
plt.grid()
plt.show()

(26,)
[0]
[25]


RuntimeError: Newton solver did not converge because maximum number of iterations reached